# AI Assignment: Footfall Counter using Computer Vision
**Objective:**  
Develop a computer vision–based system that counts the number of people entering and exiting through a specific area using object detection and tracking.

**Student Name:** Mithun Piriyan  
**Tools Used:** Python, YOLOv8, OpenCV, Numpy


In [3]:
!pip install ultralytics opencv-python numpy matplotlib --quiet


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\mithunn\lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [5]:
# Load YOLOv8 model (Nano version is light and fast)
model = YOLO("yolov8n.pt")  # Automatically downloads model if not found

print("✅ YOLOv8 model loaded successfully!")


NameError: name 'YOLO' is not defined

In [6]:
def get_line(frame):
    """Returns the y-coordinate for the counting line."""
    h, w, _ = frame.shape
    return h // 2  # middle of the frame


In [7]:
enter_count = 0
exit_count = 0
track_memory = {}


In [8]:
# Open webcam (0 = default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not open webcam.")
else:
    print("✅ Webcam started successfully!")


✅ Webcam started successfully!


In [9]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Frame read failed. Exiting...")
        break

    y_line = get_line(frame)
    
    # Run YOLO detection + tracking
    results = model.track(frame, persist=True, show=False, classes=[0])  # class 0 = person

    # Draw ROI (counting line)
    cv2.line(frame, (0, y_line), (frame.shape[1], y_line), (0, 255, 255), 2)

    if results[0].boxes.id is not None:
        for box, track_id in zip(results[0].boxes.xyxy, results[0].boxes.id):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

            prev_y = track_memory.get(track_id, cy)
            track_memory[track_id] = cy

            # Count crossing logic
            if prev_y < y_line and cy >= y_line:
                enter_count += 1
            elif prev_y > y_line and cy <= y_line:
                exit_count += 1

            # Draw boxes and ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"ID {int(track_id)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display counts
    cv2.putText(frame, f"Entered: {enter_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 3)
    cv2.putText(frame, f"Exited: {exit_count}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

    # Show frame
    cv2.imshow("Footfall Counter", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'model' is not defined

In [10]:
## ✅ Results
- Real-time detection and counting performed successfully.
- Each person crossing the defined ROI line is counted as **Entry** or **Exit**.
- Counts are displayed dynamically on the video frame.


SyntaxError: invalid syntax (3973052422.py, line 2)

In [11]:
Below is an example output frame:

![Output Sample](output_sample.png)


SyntaxError: invalid syntax (3529970766.py, line 1)

### References
- [Ultralytics YOLOv8 Documentation](https://docs.ultralytics.com)
- [OpenCV Python Docs](https://docs.opencv.org)
